In [ ]:
import numpy as np
import xarray as xr
import os
import rasterio
from rasterio.transform import Affine
from scipy.interpolate import RegularGridInterpolator

In [ ]:
def geotiff_to_dataarray(geotiff_path: str, band: int = 1) -> xr.DataArray:
    """
    Lê um GeoTIFF (single- or multi-band) como um xarray.DataArray.

    Parâmetros
    ----------
    geotiff_path : str
        Caminho do arquivo GeoTIFF.
    band : int, opcional (default=1)
        Índice da banda a ler (1-based, como no rasterio).

    Retorna
    -------
    xr.DataArray
        DataArray com dims ('y','x') e coords 1D 'y' (lat) e 'x' (lon), quando não há rotação.
        Se houver rotação no transform, retorna coords 2D ('y','x') com arrays 'x' e 'y' bidimensionais.
    """
    if not os.path.exists(geotiff_path):
        raise FileNotFoundError(f"File not found: {geotiff_path}")

    with rasterio.open(geotiff_path) as src:
        arr = src.read(band, masked=True)          # masked array (respeita nodata)
        transform: Affine = src.transform
        crs = src.crs
        nodata = src.nodata
        height, width = src.height, src.width

    # Converte nodata -> NaN (mantém float)
    data = np.asarray(arr.filled(np.nan), dtype=float)

    # Componentes do affine: [a, b, c; d, e, f]
    a, b, c, d, e, f = transform.a, transform.b, transform.c, transform.d, transform.e, transform.f

    # Caso comum (sem rotação): b == 0 e d == 0  -> coords 1D
    if np.isclose(b, 0.0) and np.isclose(d, 0.0):
        # centros de pixel: origem é canto sup-esq (c,f)
        dx, dy = a, e  # dy costuma ser negativo em GeoTIFF norte-acima
        x = c + (np.arange(width) + 0.5) * dx
        y = f + (np.arange(height) + 0.5) * dy

        da = xr.DataArray(
            data,
            dims=("y", "x"),
            coords={"y": y, "x": x},
            name="band1",
            attrs={
                "crs": crs.to_string() if crs is not None else None,
                "transform": (a, b, c, d, e, f),
                "nodata": nodata,
            },
        )

    else:
        # Transformação com rotação -> coordenadas dependem de linha/coluna
        rows = np.arange(height)
        cols = np.arange(width)
        # grade de índices
        C, R = np.meshgrid(cols, rows)
        # centros de pixel: (col+0.5, row+0.5)
        X = a*(C+0.5) + b*(R+0.5) + c
        Y = d*(C+0.5) + e*(R+0.5) + f

        da = xr.DataArray(
            data,
            dims=("y", "x"),
            coords={"y": (("y","x"), Y), "x": (("y","x"), X)},
            name="band1",
            attrs={
                "crs": crs.to_string() if crs is not None else None,
                "transform": (a, b, c, d, e, f),
                "nodata": nodata,
            },
        )

    # Garante eixo y crescente (sul→norte) — se vier decrescente, inverte dados e coords
    if np.ndim(da.coords["y"]) == 1 and da.y.size > 1 and da.y[0] > da.y[-1]:
        da = da.isel(y=slice(None, None, -1))
    elif np.ndim(da.coords["y"]) == 2:
        # 2D: ordena linhas se necessário
        if da.y[0,0] > da.y[-1,0]:
            da = da.isel(y=slice(None, None, -1))

    return da

In [ ]:
# --- Parâmetros principais (iguais ao .m) ---
resolucao_graus = 0.05       # Δλ = Δφ (graus)
raio_integracao_graus = 1.0  # raio de integração (graus)
grau_modificacao = 300       # truncamento da série

In [ ]:
# --- 1) Ler grade residual (mGal) como xarray.DataArray ---
da_res = geotiff_to_dataarray("../data/processed/Grade_AG_res_Poly9.tif")

# Checagens rápidas (opcional)
print("dims:", da_res.dims, "shape:", da_res.shape)
print("lat range:", float(da_res.y.min()), "→", float(da_res.y.max()))
print("lon range:", float(da_res.x.min()), "→", float(da_res.x.max()))

In [ ]:
# --- 2) Converter valores de mGal para m/s² (equivale ao *1D-5 do MATLAB) ---
Ag_res_mps2 = da_res.values.astype(float) * 1e-5  # m/s²

# --- 3) Eixos (graus) e limites ---
lat_vec_deg = np.asarray(da_res.y.values, dtype=float)   # S→N (crescente)
lon_vec_deg = np.asarray(da_res.x.values, dtype=float)   # W→E (crescente)

lin, col = Ag_res_mps2.shape
latS, latN = float(lat_vec_deg.min()), float(lat_vec_deg.max())
lonW, lonE = float(lon_vec_deg.min()), float(lon_vec_deg.max())

# --- 4) Grades Q (integração) em radianos (igual ao grade_lat/grade_long do MATLAB) ---
grade_lat_rad  = np.deg2rad(np.tile(lat_vec_deg[:, None], (1, col)))  # (lin, col)
grade_long_rad = np.deg2rad(np.tile(lon_vec_deg[None, :], (lin, 1)))  # (lin, col)

# Vetores "achatados" dos pontos Q (integração)
latQ = grade_lat_rad.ravel()
longQ = grade_long_rad.ravel()
AgQ = Ag_res_mps2.ravel()
sin_latQ = np.sin(latQ)
cos_latQ = np.cos(latQ)

In [ ]:
# --- 5) Domínio dos pontos P (cálculo): recua 1° em todos os lados, como no .m ---
latP_N = latN - raio_integracao_graus   # latitude norte da grade P
lonP_W = lonW + raio_integracao_graus   # longitude oeste da grade P
latP_S = latS + raio_integracao_graus   # latitude sul da grade P
lonP_E = lonE - raio_integracao_graus   # longitude leste da grade P

# Seleciona valores de lat/lon P exatamente na malha da imagem, dentro do domínio recuado
latP_vals_deg = lat_vec_deg[(lat_vec_deg >= latP_S - 1e-12) & (lat_vec_deg <= latP_N + 1e-12)]
lonP_vals_deg = lon_vec_deg[(lon_vec_deg >= lonP_W - 1e-12) & (lon_vec_deg <= lonP_E + 1e-12)]

if latP_vals_deg.size == 0 or lonP_vals_deg.size == 0:
    raise ValueError("Domínio P ficou vazio — confira raio_integracao_graus/resolução.")

# Grade P (em radianos)
grade_latP_rad  = np.deg2rad(np.tile(latP_vals_deg[:, None], (1, lonP_vals_deg.size)))
grade_longP_rad = np.deg2rad(np.tile(lonP_vals_deg[None, :], (latP_vals_deg.size, 1)))

latP = grade_latP_rad.ravel()
longP = grade_longP_rad.ravel()

# --- 6) Relatório rápido para garantir que está consistente ---
print(f"Q: {latQ.size} pontos (integração) | P: {latP.size} pontos (cálculo)")
print(f"P grid shape: ({latP_vals_deg.size}, {lonP_vals_deg.size})  [lat x lon]")

In [ ]:
# --- Interpolação de Δg nos P (bilinear: estável e suficiente) ---
interp = RegularGridInterpolator(
    (np.deg2rad(lat_vec_deg), np.deg2rad(lon_vec_deg)),
    Ag_res_mps2,                  # Δg(Q) em m/s²
    method="linear",
    bounds_error=False,
    fill_value=np.nan,
)

AgP = interp(np.column_stack([latP, longP]))  # Δg(P) em m/s²

# --- Gravidade normal no elipsoide em P (mesma expressão do .m) ---
sin_latP = np.sin(latP)
gama0 = 9.7803267715 * (
    1
    + 0.0052790414 * (sin_latP**2)
    + 0.0000232718 * (sin_latP**4)
    + 0.0000001262 * (sin_latP**6)
    + 0.0000000007 * (sin_latP**8)
)  # m/s²

# --- Constantes geométricas ---
res_rad = np.deg2rad(resolucao_graus)
raio_integ_rad = np.deg2rad(raio_integracao_graus)
R = 6_371_000.0
res_rad_div_2 = 0.5 * res_rad

# cte = 4*pi*R*gamma0 (vetor em P, como no .m)
cte = 4 * np.pi * R * gama0

# Termos de modificação: n = 2..grau_modificacao
n_mod = np.arange(2, grau_modificacao + 1)
cte_mod = (2 * n_mod + 1) / (n_mod - 1)

# --- Áreas dos pixels Q (Ak), em m², seguindo o truque da prática ---
Ak = np.abs((R**2) * res_rad * (np.sin(latQ + res_rad_div_2) - np.sin(latQ - res_rad_div_2)))

# --- Contribuição interna (Ni), em metros ---
Ni = (R / gama0) * np.sqrt((np.cos(latP) * (res_rad**2)) / np.pi) * AgP

# --- Checagens rápidas ---
print(f"AgP (m/s²): min={np.nanmin(AgP):.3e}, max={np.nanmax(AgP):.3e}")
print(f"gamma0 (m/s²): min={gama0.min():.6f}, max={gama0.max():.6f}")
print(f"Ak (m²): min={Ak.min():.3e}, max={Ak.max():.3e}")
print(f"Ni (m): min={np.nanmin(Ni):.3e}, max={np.nanmax(Ni):.3e}")


In [ ]:
# --- Legendre ordem 0 (igual ao do MATLAB) ---
def legendreN(nmax: int, t: np.ndarray) -> np.ndarray:
    """
    Retorna array (len(t), nmax+1) com P_0..P_nmax não-normalizados.
    Recorrência: P0=1, P1=t, Pn=((2n-1)/n)*t*P_{n-1} - ((n-1)/n)*P_{n-2}
    """
    t = np.asarray(t, dtype=float)
    P = np.empty((t.size, nmax + 1), dtype=float)
    P[:, 0] = 1.0
    if nmax >= 1:
        P[:, 1] = t
    for n in range(2, nmax + 1):
        P[:, n] = ((2*n - 1)/n) * t * P[:, n-1] - ((n - 1)/n) * P[:, n-2]
    return P

# --- Integração ponto a ponto ---
N_res = np.zeros(latP.size, dtype=float)
eps = 1e-15

# pré-calcular só pra clareza (já temos sin/cos de latQ)
cos_latQ = np.cos(latQ)  # se não existir no teu escopo, já foi feito na Etapa 1
sin_latQ = np.sin(latQ)

for p in range(latP.size):
    cos_latP_p = np.cos(latP[p])
    sin_latP_p = np.sin(latP[p])

    # 1) ângulo P-Q
    cos_psi = sin_latQ * sin_latP_p + cos_latQ * cos_latP_p * np.cos(longQ - longP[p])
    psi = np.arccos(np.clip(cos_psi, -1.0, 1.0))

    # filtra pelo raio
    m = psi <= raio_integ_rad
    if not np.any(m):
        N_res[p] = Ni[p]  # só a zona interna
        continue

    psi_int     = psi[m]
    latQ_int    = latQ[m]
    longQ_int   = longQ[m]
    AgQ_int     = AgQ[m]
    Ak_int      = Ak[m]
    cos_psi_int = cos_psi[m]

    # 2) remover singularidade (próprio pixel de P)
    k_lat = np.argmin(np.abs(latP[p]  - latQ_int))
    k_lon = np.argmin(np.abs(longP[p] - longQ_int))
    idx = np.where((np.abs(latQ_int - latQ_int[k_lat]) < 1e-14) &
                   (np.abs(longQ_int - longQ_int[k_lon]) < 1e-14))[0]
    if idx.size == 0:
        idx = [np.argmin(np.hypot(latQ_int - latP[p], longQ_int - longP[p]))]
    pos_p = idx[0]

    # 3) S(psi) mod. Wong & Gore: usa s = sin(Δ/2) por componentes (como no MATLAB)
    s2 = (np.sin((latP[p]  - latQ_int) / 2.0)**2
          + (np.sin((longP[p] - longQ_int) / 2.0)**2) * cos_latP_p * np.cos(latQ_int))
    s  = np.sqrt(s2)
    S_psi = 1.0/np.maximum(s, eps) - 4.0 - 6.0*s + 10.0*s2 - (3.0 - 6.0*s2) * np.log(s2 + s + eps)
    S_psi[pos_p] = np.nan  # remove o infinito do próprio pixel

    # 4) Série harmônica truncada (graus 2..N)
    Pn = legendreN(grau_modificacao, cos_psi_int)  # (M, nmax+1)
    S_psi_SH = Pn[:, 2:] @ cte_mod                 # (M,)

    # 5) Integração discreta + contribuição interna
    integrando = (S_psi - S_psi_SH) * Ak_int * AgQ_int
    N_res[p] = (np.nansum(integrando) / cte[p]) + Ni[p]

# Empacotar em grade 2D (lat x lon) usando os eixos P:
nlin = latP_vals_deg.size
ncol = lonP_vals_deg.size
N_grid = N_res.reshape(nlin, ncol)

da_Nres = xr.DataArray(
    N_grid,
    dims=("y", "x"),
    coords={"y": latP_vals_deg, "x": lonP_vals_deg},
    name="N_res_m",
    attrs={"units": "m", "description": "Altura geoidal residual (Stokes mod.)"},
)

# Relatório rápido
print("N_res stats (m): min={:.4f}, max={:.4f}, mean={:.4f}".format(
    np.nanmin(N_res), np.nanmax(N_res), np.nanmean(N_res)
))
da_Nres


In [ ]:
import matplotlib.pyplot as plt

# 1) Mapa com colormap
plt.figure(figsize=(10, 6))
da_Nres.plot(
    cmap="RdBu_r",  # azul-negativo, vermelho-positivo
    cbar_kwargs={"label": "N_res (m)"}
)
plt.title("Modelo geoidal residual (Stokes mod.)")
plt.xlabel("Longitude (°)")
plt.ylabel("Latitude (°)")
plt.show()

# 2) Histograma dos valores
plt.figure(figsize=(8, 4))
plt.hist(N_res[~np.isnan(N_res)], bins=50, color="gray", edgecolor="black")
plt.xlabel("N_res (m)")
plt.ylabel("Frequência")
plt.title("Distribuição dos valores de N_res")
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

# 3) Perfil em uma linha central (lat ~ mediana)
lat_central = latP_vals_deg[len(latP_vals_deg)//2]
perfil = da_Nres.sel(y=lat_central, method="nearest")

plt.figure(figsize=(10, 4))
plt.plot(perfil["x"], perfil.values, marker="o", lw=1)
plt.xlabel("Longitude (°)")
plt.ylabel("N_res (m)")
plt.title(f"Perfil N_res na latitude {lat_central:.2f}°")
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()


In [ ]:
def dataarray_to_geotiff(da, out_path, nodata=np.nan, compress="LZW"):
    """
    Exporta um xarray.DataArray 2D (lat/lon) para GeoTIFF em EPSG:4326.

    Parâmetros
    ----------
    da : xr.DataArray
        Deve ter dims ('y', 'x') em graus.
    out_path : str
        Caminho do arquivo GeoTIFF de saída.
    nodata : float
        Valor para marcar NoData (default = np.nan).
    compress : str
        Algoritmo de compressão GTiff (ex: "LZW", "DEFLATE").

    """
    from rasterio.transform import from_origin
    y = da.y.values
    x = da.x.values
    dy = float(np.abs(np.diff(y)).mean())
    dx = float(np.abs(np.diff(x)).mean())

    # Rasterio espera origem no canto superior esquerdo
    transform = from_origin(x.min() - dx/2, y.max() + dy/2, dx, dy)

    arr = np.flipud(da.values.astype("float32"))  # flip Y para "top-down"

    with rasterio.open(
        out_path,
        "w",
        driver="GTiff",
        height=arr.shape[0],
        width=arr.shape[1],
        count=1,
        dtype="float32",
        crs="EPSG:4326",
        transform=transform,
        nodata=nodata,
        compress=compress,
        tiled=True,
        predictor=3,
    ) as dst:
        dst.write(arr, 1)

    print(f"[OK] Exportado GeoTIFF: {out_path}")


# salva em ../data/processed/N_residual.tif (EPSG:4326)
dataarray_to_geotiff(da_Nres, "../data/processed/N_residual.tif")